In [ ]:
import json, os
import pandas as pd
from matplotlib import pyplot as plt
from collections import defaultdict
import numpy as np
from itertools import combinations
from sklearn.linear_model import LinearRegression, RANSACRegressor
from sklearn.decomposition import PCA
from wpca import WPCA
from sklearn.preprocessing import StandardScaler
from aquabyte.accuracy_metrics import AccuracyMetricsGenerator
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from aquabyte.optics import euclidean_distance, pixel2world
from aquabyte.visualize import Visualizer
import random
from scipy.stats import norm
from PIL import Image, ImageDraw
from urllib.parse import urlparse

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)

<h1> Get lice annotation data </h1>

In [ ]:
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))
query = """
    select * from lati_fish_detections_lice_annotations where (pen_id = 57 or pen_id=58 or pen_id=59 or pen_id=60)
    and captured_at >= '2019-09-07'
"""
df = rds_access_utils.extract_from_database(query)

In [ ]:
query = """
    select * from lati_fish_detections_lice_annotations_reconciled where (pen_id = 57 or pen_id=58 or pen_id=59 or pen_id=60)
    and captured_at >= '2019-09-07'
"""
reconciled_df = rds_access_utils.extract_from_database(query)

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')

In [ ]:
df = df[df.is_cleaner_fish != True]

In [ ]:
def depth_fn(x):
    w, h = x['width'], x['height']
    theta = np.arctan(h / w) * (180.0 / np.pi)
    phi = np.arctan(0.25) * (180.0 / np.pi)
    if theta < phi:
        return w
    elif theta > 90.0 - phi:
        return h
    else:
        return (h**2 + w**2)**0.5

In [ ]:
focal_length_px = 4015
avg_fish_length_cm = 25.0

df['image_width'] = df.metadata.apply(lambda x: x['width'])
df['image_height'] = df.metadata.apply(lambda x: x['height'])
df['length_px'] = df.metadata.apply(lambda x: depth_fn(x))
df['depth'] = focal_length_px * avg_fish_length_cm / df.length_px

In [ ]:
focal_length_px = 4015
avg_fish_length_cm = 25.0

reconciled_df['image_width'] = reconciled_df.metadata.apply(lambda x: x['width'])
reconciled_df['image_height'] = reconciled_df.metadata.apply(lambda x: x['height'])
reconciled_df['length_px'] = reconciled_df.metadata.apply(lambda x: depth_fn(x))
reconciled_df['depth'] = focal_length_px * avg_fish_length_cm / reconciled_df.length_px

In [ ]:
in_focus_mask = (df.depth > 75) & (df.depth < 105)
cogito_accept_mask = ~df.is_skipped


n = df.shape[0]
n_in_focus = df[in_focus_mask].shape[0]
n_in_focus_accepted_cogito = df[in_focus_mask & (cogito_accept_mask)].shape[0]
n_not_in_focus_accepted_cogito = df[~in_focus_mask & (cogito_accept_mask)].shape[0]


print('Total number of images inspected by Cogito over the weekend: {}'.format(n))
print('Total number of these images within in-focus range (75 cm - 105 cm): {}'.format(n_in_focus))
print('Total number of in-focus images accepted by Cogito: {}'.format(n_in_focus_accepted_cogito))
print('Total number of not-in-focus images accepted by Cogito: {}'.format(n_not_in_focus_accepted_cogito))

<h1> What should Cogito have done? </h1>

In [ ]:
df[in_focus_mask & cogito_accept_mask].depth

In [ ]:
for idx, row in df[in_focus_mask].iterrows():
    s3_access_utils.download_from_s3()

In [ ]:
df[~in_focus_mask & cogito_accept_mask][['image_url', 'is_too_dark', 'is_blurry', 'is_bad_crop', 'is_cleaner_fish', 'depth']]

In [ ]:
plt.hist(df[cogito_accept_mask].depth, bins=20)
plt.show()

In [ ]:
(1034**2+727**2)**.5

In [ ]:
4015*250 / (1263)

In [ ]:
(4015 * 0.005) / 0.78

In [ ]:
reconciled_df[reconciled_df.adult_female_count > 0].head()

In [ ]:
reconciled_df[['adult_female_count_adjusted', 'moving_count_adjusted']].sum()

In [ ]:
reconciled_df[reconciled_df.moving_count_adjusted == 1]

In [ ]:
focal_length = 0.0138 / 3.45e-6
baseline = 0.101
disparity = (248-200) * (4096/512.)
depth = focal_length * baseline / disparity
print(depth)

In [ ]:
plt.hist(reconciled_df[~reconciled_df.is_skipped].depth, bins=20)
plt.show()

In [ ]:
reconciled_df.loc[~reconciled_df.is_skipped, ['image_url', 'depth']]

In [ ]:
df[df.is_skipped == True].sample(2000)

In [ ]:
modified_images_dir = '/root/data/alok/biomass_estimation/modified_images/'
object_length_cm = 1.0

cogito_accept_mask = ~df.is_skipped
qa_accept_mask = ~reconciled_df.is_skipped
depth_values = np.arange(25, 125, 10)
for i in range(len(depth_values)-1):
    lo, hi = int(depth_values[i]), int(depth_values[i+1])
    depth_mask = (df.depth >= lo) & (df.depth <= hi)
    reconciled_depth_mask = (reconciled_df.depth >= lo) & (reconciled_df.depth <= hi)
    
    # rejected images
    for idx, row in df[depth_mask & ~cogito_accept_mask].head(20).iterrows():
        depth = row.depth
        line_segment_length_px = object_length_cm * 4015 / depth
        image_url = row.image_url
        s3_path_components = urlparse(image_url, allow_fragments=False).path.lstrip('/').split('/')
        bucket, key = s3_path_components[0], os.path.join(*s3_path_components[1:])
        image_f = s3_access_utils.download_from_s3(bucket, key)
        
        im = Image.open(image_f)
        draw = ImageDraw.Draw(im)
        draw.line((100, 100, 100+line_segment_length_px, 100))
        
        f_name = os.path.basename(key)
        f = os.path.join(modified_images_dir, '{}_{}'.format(lo, hi), 'rejected', f_name)
        if not os.path.exists(os.path.dirname(f)):
            os.makedirs(os.path.dirname(f))
        im.save(f)
        break
        
    # accepted images
    for idx, row in reconciled_df[reconciled_depth_mask & qa_accept_mask].iterrows():
        depth = row.depth
        line_segment_length_px = object_length_cm * 4015 / depth
        image_url = row.image_url
        s3_path_components = urlparse(image_url, allow_fragments=False).path.lstrip('/').split('/')
        bucket, key = s3_path_components[0], os.path.join(*s3_path_components[1:])
        image_f = s3_access_utils.download_from_s3(bucket, key)
        
        im = Image.open(image_f)
        draw = ImageDraw.Draw(im)
        draw.line((100, 100, 100+line_segment_length_px, 100))
        
        f_name = os.path.basename(key)
        f = os.path.join(modified_images_dir, '{}_{}'.format(lo, hi), 'accepted', f_name)
        if not os.path.exists(os.path.dirname(f)):
            os.makedirs(os.path.dirname(f))
        im.save(f)
        break

        

<h1> Generate depth values </h1>